# Question 1

## Install Packages for the read_html

In [1]:
!pip install pandas
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

     |████████████████████████████████| 5.5MB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 122kB 6.4MB/s eta 0:00:01


## Read the table from the wikipedia into a data frame

In [2]:
import pandas as pd
import html5lib
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Drop all rows with Boroughs labeled Not assigned

In [4]:
df=df[df.Borough != "Not assigned"]
df.reset_index(drop=True, inplace=True)

df.shape


(103, 3)

In [5]:
!pip install numpy


## All Boroughs with neighborhoods labeled as Not assigned will then have the neirborhood labeled as the Borough

In [6]:
import numpy as np
df['Neighborhood'] = np.where( (df['Neighborhood'] =='Not assigned'), df['Borough'],df['Neighborhood'])      #(looking for, changed to, affected)
df.shape

(103, 3)

## Combine all Neighborhoods into same row if they have the same Postal code

In [7]:
df=df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df.shape

(103, 3)

# Question 2

## Create another df with all the postal codes latitudes and longitudes

In [15]:
ll=pd.read_csv('https://cocl.us/Geospatial_data')
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge the original df with the postal code df based on the common column called postal code

In [16]:
full=pd.merge(df,ll,on='Postal Code')
full.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
full.shape

(103, 5)

# Question 3

## Install needed packages

In [25]:
!pip install geopy 
!pip install folium

## Get the coordinates of Toronto Canada

In [22]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create a map of Toronto with the neighborhoods marked

In [26]:
import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(full['Latitude'], full['Longitude'], full['Borough'], full['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Credentials needed for Foursquare

In [27]:
CLIENT_ID = '035VRQJS0CKYBIGLEWT24OX2D2TORBSOKTC3A4SKBP5WTKTS' # your Foursquare ID
CLIENT_SECRET = 'YQSGM2GNIDUHPIKRHKRVO2A03C3N43DU2GH4Q0EXLCR1XEBS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 035VRQJS0CKYBIGLEWT24OX2D2TORBSOKTC3A4SKBP5WTKTS
CLIENT_SECRET:YQSGM2GNIDUHPIKRHKRVO2A03C3N43DU2GH4Q0EXLCR1XEBS


## Create a function that gets nearby venues for each lat/long

In [36]:
import requests # library to handle requests
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Called the function to run on the toronto data

In [37]:
toronto_venues = getNearbyVenues(names=full['Borough'],
                                   latitudes=full['Latitude'],
                                   longitudes=full['Longitude']
                                  )


Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [39]:
toronto_venues.head()

,Borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Scarborough,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


## Merged the venue data with the postal code data

In [46]:
total=full.merge(toronto_venues,on=['Borough','Latitude','Longitude'])
total.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [47]:
total.groupby('Neighborhood').count()

,Postal Code,Borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,,
Agincourt,5,5,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23,23,23
...,...,...,...,...,...,...,...,...
"Willowdale, Willowdale West",7,7,7,7,7,7,7,7
Woburn,3,3,3,3,3,3,3,3
Woodbine Heights,9,9,9,9,9,9,9,9


## Found how many of each venue type there is to each Neighborhood/zipcode

In [57]:
# one hot encoding
toronto_onehot = pd.get_dummies(total[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood']=total['Neighborhood']

#Neighborhood the first column
col_name="Neighborhood"
first_col = toronto_onehot.pop(col_name)
toronto_onehot.insert(0, col_name, first_col)

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Descriptive data for each neighborhood

In [58]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Create a function that returns the most common venues in an area

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Finds the top ten venues in each Neighborhood/zipcode

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Latin American Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Coffee Shop,Sandwich Place,Skating Rink,Pub,Yoga Studio,Discount Store,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Supermarket,Bridal Shop,Shopping Mall,Sandwich Place,Restaurant,Pizza Place,Mobile Phone Shop
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Pharmacy,Sushi Restaurant,Grocery Store,Indian Restaurant,Juice Bar,Liquor Store


## Clusters the neighborhoods/zipcodes into five clusters

In [68]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 1, 1, 1, 3, 1, 3, 1], dtype=int32)

## Merge the top ten venues with our zip code data

In [70]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = full

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,College Rec Center
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,3.0,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Mexican Restaurant,Breakfast Spot,Electronics Store,Intersection,Medical Center,Bank,Rental Car Location,Dog Run,Discount Store,Distribution Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Gas Station,Bakery,Discount Store


## Clean up the merged data 
Some zip codes do not have clusters due to having no venues so they need to be dropped before running the next portion

In [81]:
toronto_merged['Cluster Labels'].isnull().values.any()

True

In [83]:
toronto_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)
toronto_merged['Cluster Labels'].isnull().values.any()

False

## Create a map of Toronto with the neighborhoods and the clusters color coded

In [85]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np 

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=(rainbow[int(cluster)-1]),
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1]+[2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",0.0,Playground,Park,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
20,North York,"York Mills, Silver Hills",0.0,Cafeteria,Park,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
23,North York,York Mills West,0.0,Park,Bar,Convenience Store,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
25,North York,Parkwoods,0.0,Food & Drink Shop,Park,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
40,East York,"East Toronto, Broadview North (Old East York)",0.0,Park,Convenience Store,Coffee Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
44,Central Toronto,Lawrence Park,0.0,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
50,Downtown Toronto,Rosedale,0.0,Park,Trail,Playground,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
74,York,Caledonia-Fairbanks,0.0,Park,Women's Store,Pool,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
90,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",0.0,River,Park,Smoke Shop,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
98,York,Weston,0.0,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


### Cluster 2

In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1]+[2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,Woburn,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
6,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",1.0,Discount Store,Train Station,Coffee Shop,Bus Station,Department Store,Yoga Studio,Dim Sum Restaurant,Diner,Distribution Center,Dog Run
10,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",1.0,Indian Restaurant,Light Rail Station,Chinese Restaurant,Vietnamese Restaurant,Pet Store,Brewery,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
15,Scarborough,"Steeles West, L'Amoreaux West",1.0,Coffee Shop,Chinese Restaurant,Breakfast Spot,Sandwich Place,Thrift / Vintage Store,Fast Food Restaurant,Bank,Pizza Place,Pharmacy,Grocery Store
18,North York,"Fairview, Henry Farm, Oriole",1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Restaurant,Bank,Bakery,Shoe Store,Japanese Restaurant,Tea Room
19,North York,Bayview Village,1.0,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant
22,North York,"Willowdale, Willowdale East",1.0,Ramen Restaurant,Restaurant,Sandwich Place,Café,Coffee Shop,Pizza Place,Indonesian Restaurant,Sushi Restaurant,Ice Cream Shop,Steakhouse
24,North York,"Willowdale, Willowdale West",1.0,Pizza Place,Pharmacy,Grocery Store,Bank,Butcher,Coffee Shop,Discount Store,Dog Run,Dim Sum Restaurant,Diner
26,North York,Don Mills,1.0,Gym,Asian Restaurant,Restaurant,Japanese Restaurant,Coffee Shop,Beer Store,Discount Store,Café,Italian Restaurant,Dim Sum Restaurant
27,North York,Don Mills,1.0,Gym,Asian Restaurant,Restaurant,Japanese Restaurant,Coffee Shop,Beer Store,Discount Store,Café,Italian Restaurant,Dim Sum Restaurant


### Cluster 3

In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1]+[2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",2.0,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
97,North York,"Humberlea, Emery",2.0,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop


### Cluster 4

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1]+[2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",3.0,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,College Rec Center
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",3.0,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
2,Scarborough,"Guildwood, Morningside, West Hill",3.0,Mexican Restaurant,Breakfast Spot,Electronics Store,Intersection,Medical Center,Bank,Rental Car Location,Dog Run,Discount Store,Distribution Center
4,Scarborough,Cedarbrae,3.0,Bank,Lounge,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Gas Station,Bakery,Discount Store
5,Scarborough,Scarborough Village,3.0,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,College Gym
7,Scarborough,"Golden Mile, Clairlea, Oakridge",3.0,Bus Line,Bakery,Soccer Field,Ice Cream Shop,Bus Station,Metro Station,Park,Dumpling Restaurant,Drugstore,Donut Shop
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",3.0,American Restaurant,Motel,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
9,Scarborough,"Birch Cliff, Cliffside West",3.0,College Stadium,Café,Skating Rink,General Entertainment,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
11,Scarborough,"Wexford, Maryvale",3.0,Breakfast Spot,Bakery,Smoke Shop,Middle Eastern Restaurant,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Drugstore
12,Scarborough,Agincourt,3.0,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Latin American Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant


### Cluster 5

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1]+[2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,North York,Humber Summit,4.0,Pizza Place,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,College Rec Center
